# Scraper des Discours

This notebook is foreked from:
https://github.com/Araxide/french_speech_scraper/blob/master/discours_scraper.py
I have updated the calls selenium's find_elements_by* functions.
Many thanks!

In [1]:
!pip install webdriver-manager

In [2]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 14.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [30]:
import csv
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from selenium.webdriver.common.by import By


In [36]:
class DiscoursScraper():
  
    def __init__(self):
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        # à changer si tu as des critères de recherche
        self.main_page_url = 'https://www.vie-publique.fr/discours/recherche?search_api_fulltext_discours=&sort_by=field_date_prononciation_discour&field_intervenant_title=&field_intervenant_qualite=&field_date_prononciation_discour_interval%5Bmin%5D=&field_date_prononciation_discour_interval%5Bmax%5D=&field_type_emetteur=&field_type_de_document='
        self.child_page_url = 'https://www.vie-publique.fr/discours/recherche?search_api_fulltext_discours=&sort_by=field_date_prononciation_discour&field_intervenant_title=&field_intervenant_qualite=&field_date_prononciation_discour_interval%5Bmin%5D=&field_date_prononciation_discour_interval%5Bmax%5D=&field_type_emetteur=&field_type_de_document=&page=1'
        # à changer selon vos besoins
        self.pages_begin = 1
        self.pages_end = 12 #100 

    def scrap_speeches_urls(self, page_url):
        self.driver.get(page_url)
        sleep(3) # wait until all elements have loaded
        articles = self.driver.find_elements(By.CSS_SELECTOR, "h3.fr-card__title > a")
        articles = [article.get_attribute('href') for article in articles]
        return articles

    def record_speeches_of_page(self, writer, speeches_urls):
        for speech_url in speeches_urls:
            self.driver.get(speech_url)
            try:                
                sleep(2) # wait until all elements have loaded
                discours = self.driver.find_elements(By.CSS_SELECTOR, "div.vp-discours-content > div > p")
                discoursText = ""
                for d in discours:
                    discoursText += self.remove_line_breaks(d)
                # discours title  
                title = self.driver.find_element(By.CSS_SELECTOR, "h1")
                # discours date
                date_discours = self.driver.find_element(By.CSS_SELECTOR, "time")
                # append row on csv file
                writer.writerow({'titre': title.text, 'date': date_discours.text, 'discours': discoursText})
            except (NoSuchElementException):
                pass #pour l'instant on l'ignore
    
    def remove_line_breaks(self, element):
        element = element.text.replace('\n', ' ').replace('\r', '') # remove line breaks in text
        return str(element)
  
    def main(self):
        # create csv
        with open('scraped_discours.csv', 'a', encoding='utf-8', newline='') as csvfile:
            fieldnames = ['titre', 'date', 'discours']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            # scrap first page -- à enlever si on ne commence pas par le page 0
            speeches_urls_first_page = self.scrap_speeches_urls(self.main_page_url)
            self.record_speeches_of_page(writer, speeches_urls_first_page)      
            
            # get speeches urls for each next page until reach pages_total
            for i in range(self.pages_begin, self.pages_end):
                speeches_urls = self.scrap_speeches_urls(self.child_page_url+str(i))
                # scrap speech text
                self.record_speeches_of_page(writer, speeches_urls)

        # end webdriver session    
        self.driver.quit()

In [37]:
DiscoursScraper().main()

/tmp/ipykernel_316831/3562880954.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())
